In [35]:
import cv2 as cv
import numpy as np
import os





In [36]:
path_folder_joc="../../../antrenare"
path_backgroud="../../../imagini_auxiliare/01.jpg"
path_solutie="341_Balaita_Cosmin"

In [37]:
def show_image(title,image):
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

Extragere Tabla de joc de pe masa


In [38]:
def extrage_tabla(image):
    image= cv.cvtColor(image,cv.COLOR_BGR2HSV) #convertim imaginea in hsv pentru a putea extrage tabla de joc
    h,s,v = cv.split(image)

    image_sharp = cv.medianBlur(h,3)
    image_blur = cv.GaussianBlur(image_sharp, (0, 0), 5)
    image_sharp=cv.addWeighted(image_sharp, 1.5, image_blur, -0.5, 0)
    _,thresh = cv.threshold(image_sharp, 30, 255, cv.THRESH_BINARY)
    kernel = np.ones((3,3),np.uint8)
    tresh = cv.erode(thresh,kernel)
    edges = cv.Canny(thresh,300,400)

    contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE) #extragem contururile
    max_area = 0 #initializam aria maxima

    for contour in contours:
        if len(contour) > 3: #daca conturul are mai mult de 3 puncte (un patrat)
            pos_top_left = None
            pos_bottom_right = None
            for point in contour.squeeze(): #extragem cele 4 puncte ale patratului
                if pos_top_left is None or point[0] + point[1] < pos_top_left[0] + pos_top_left[1]: #
                    pos_top_left = point
                if pos_bottom_right is None or point[0] + point[1] > pos_bottom_right[0] + pos_bottom_right[1]:
                    pos_bottom_right = point
            diff = np.diff(contour.squeeze(),axis=1) #diferenta dintre coordonatele x si y
            pos_top_right = contour.squeeze()[np.argmin(diff)] #extragem coordonatele punctului din dreapta sus
            pos_bottom_left = contour.squeeze()[np.argmax(diff)] #extragem coordonatele punctului din stanga jos

            polygon = np.array([pos_top_left,pos_top_right,pos_bottom_right,pos_bottom_left],dtype="float32") #creem un poligon cu cele 4 puncte
            area = cv.contourArea(polygon) #calculam aria poligonului
            if area > max_area:
                max_area = area
                top_left = pos_top_left
                bottom_right = pos_bottom_right
                top_right = pos_top_right
                bottom_left = pos_bottom_left

    width = 810
    height = 810
    puzzle = np.array([top_left,top_right,bottom_right,bottom_left],dtype="float32") #creem un poligon cu cele 4 puncte
    destination = np.array([[0,0],[width,0],[width,height],[0,height]],dtype="float32")
    M= cv.getPerspectiveTransform(puzzle,destination) #calculam matricea de transformare
    warped = cv.warpPerspective(image,M,(width,height)) #aplicam transformarea
    result = cv.cvtColor(warped,cv.COLOR_HSV2BGR) #convertim imaginea in BGR

    return result

Extragere Careu interior

In [39]:
def extrage_careu_interior(tabla):
    height, width, _ = tabla.shape #extragem dimensiunile imaginii
    margin_top = int(height * 0.129)
    margin_bottom = int(height * 0.129)
    margin_left = int(width * 0.132)
    margin_right = int(width * 0.132)

    grid = tabla[margin_top:height - margin_bottom, margin_left:width - margin_right] #decupam zona centrala a tablei
    return grid

Extragere linii verticale si orizontale

In [40]:
def extrage_linii_verticale_si_orizontale(image): #
    height, width, _ = image.shape
    linii_verticale=[] #lista in care vom salva coordonatele liniilor verticale
    linii_orizontale=[] #lista in care vom salva coordonatele liniilor orizontale
    cell_height = height / 14
    cell_width = width / 14
    for i in range(15): #pentru liniile orizontale
        y = round(i * cell_height) #calculam coordonata y a liniei
        linii_orizontale.append([(0, y), (width, y)]) #adaugam coordonatele liniei in lista
        cv.line(image, (0, y), (width, y), (0, 0, 255), 2) #desenam linia pe imagine

    for i in range(15): #pentru liniile verticale
        x = round(i * cell_width) #calculam coordonata x a liniei
        linii_verticale.append([(x, 0), (x, height)]) #adaugam coordonatele liniei in lista
        cv.line(image, (x, 0), (x, height), (0, 0, 255), 2) #desenam linia pe imagine



    return image,linii_verticale, linii_orizontale



Determinarea configuratiei careului

In [41]:
def determina_configuratie_careu(tresh,orizontal,vertical):
    matrix = np.empty((14,14), dtype='str')
    for i in range(len(orizontal)-1):
        for j in range(len(vertical)-1):
            y_min = orizontal[j][0][1] + 20
            y_max = orizontal[j + 1][1][1] - 20
            x_min = vertical[i][0][0] + 20
            x_max = vertical[i + 1][1][0] - 20
            cell = tresh[y_min:y_max, x_min:x_max]
            if np.mean(cell) > 0:
                matrix[j][i] = '1'
            else:
                matrix[j][i] = '0'
    return matrix


Vizualizare pozitii sperciale de pe tabla

In [42]:
def vizualizare_conf(result,matrix,oriz,vert):
    for i in range(len(oriz) - 1):
        for j in range(len(vert) - 1):
            y_min = vert[j][0][0]
            y_max = vert[j + 1][1][0]
            x_min = oriz[i][0][1]
            x_max = oriz[i + 1][1][1]
            if matrix[i][j] == '1':
                cv.rectangle(result, (x_min, y_min), (x_max, y_max), (0, 255, 0), 3)

Diferenta intre doua imagini pentru a determina pe ce pozitie apare o piesa noua


In [43]:
def diff_img(imagine_veche, imagine_noua):

    imagine_veche = cv.cvtColor(imagine_veche, cv.COLOR_BGR2HSV)
    imagine_noua= cv.cvtColor(imagine_noua, cv.COLOR_BGR2HSV)
    h1,s1,v1=cv.split(imagine_veche)
    h2,s2,v2 = cv.split(imagine_noua)

    dif3=cv.absdiff(v1,v2) #diferenta dintre cele 2 imagini pe canalul
    #show_image("dif3",dif3)
    _, diff = cv.threshold(dif3, 117, 255, cv.THRESH_BINARY)

    kernel = np.ones((3,3),np.uint8)
    diff= cv.erode(diff,kernel) #erodam imaginea pentru a elimina zgomotul

    diff=cv.dilate(diff,kernel) #dilatam imaginea pentru a face piesele mai vizibile

    #show_image("diff",diff)
    return diff



Determinare pozitie piesa noua

In [44]:
def determina_pozitie_piesa_noua(imagine_veche, imagine_noua):
    diff = diff_img(imagine_veche, imagine_noua)
    _, binary_diff = cv.threshold(diff, 127, 255, cv.THRESH_BINARY)
    num_labels, labels_im = cv.connectedComponents(binary_diff) #extragem componentele conexe
    max_pixels = 0
    max_label = 0

    for label in range(1, num_labels): #parcurgem componentele conexe
        pixel_count = np.sum(labels_im == label) #calculam numarul de pixeli din componenta
        if pixel_count > max_pixels: #daca numarul de pixeli este mai mare decat cel maxim
            max_pixels = pixel_count #actualizam numarul maxim de pixeli
            max_label = label #actualizam label-ul maxim

    if max_label != 0: #daca am gasit o componenta conexa
        largest_component_mask = (labels_im == max_label).astype(np.uint8) * 255 #extragem masca componentei
        contours, _ = cv.findContours(largest_component_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE) #extragem conturul componentei

        if contours:
            cnt = contours[0] #extragem primul contur
            x, y, w, h = cv.boundingRect(cnt) #extragem dreptunghiul incadrator al conturului
            cv.rectangle(imagine_noua, (x, y), (x + w, y + h), (0, 255, 0), 2) #desenam dreptunghiul incadrator pe imagine

            return x + w // 2, y + h // 2, w, h #returnam coordonatele centrului dreptunghiului incadrator si dimensiunile acestuia

    return None


Potrivire pozitie cu matricea careului


In [45]:
def match_position(x,y,oriz,vert):
    for i in range(len(oriz)-1):
        if oriz[i][0][1] < y < oriz[i+1][1][1]: #daca y se afla intre coordonatele liniilor orizontale
            for j in range(len(vert)-1):
                if vert[j][0][0] < x < vert[j+1][1][0] : #daca x se afla intre coordonatele liniilor verticale
                    return i,j
    return -1,-1

Extrage piesa

In [46]:
def extrage_piesa (i,j,img):

    piece = img[oriz[i][0][1]:oriz[i+1][1][1]+1,vert[j][0][0]:vert[j+1][1][0]] #extragem piesa din imagine
    #show_image("piesa",piece)

    return piece



In [47]:
def classify_patch(patch):
    patch_grey=cv.cvtColor(patch,cv.COLOR_BGR2GRAY)
    #show_image("patch",patch_grey)
    patch_grey=cv.equalizeHist(patch_grey)
    #show_image("patch",patch_grey)
    _,tresh=cv.threshold(patch_grey,140,255,cv.THRESH_BINARY)
    #show_image("patch",tresh)
    #mareste patch-ul cu 20%

    max_match=0
    detected=None
    folder="templates"

    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
            img=cv.imread(os.path.join(folder,filename))
            img_grey=cv.cvtColor(img,cv.COLOR_BGR2GRAY)
            img_grey=cv.equalizeHist(img_grey)
            _,img_grey=cv.threshold(img_grey,140,255,cv.THRESH_BINARY)


            #accentueaza contururile
            #img_grey=cv.addWeighted(img_grey,1.5,img_grey,-0.5,0)
            res=cv.matchTemplate(patch_grey,img_grey,cv.TM_CCORR_NORMED)
            min_val,max_val,min_loc,max_loc=cv.minMaxLoc(res)
            if max_val>max_match:
                max_match=max_val
                detected=filename.split(".")[0]

    return detected


In [48]:
def calculeaza_scor(ture,moves,matrice_tabla,conf_speciala):
    scoruri=[]
    fplayer,fstart=ture[-1]
    for i in range(len(ture)):
        player,start = ture[i]
        if player == fplayer and start == fstart:
            stop=50
        else:
            stop=int(ture[i+1][1])


        scor=0
        nr=0
        for i in range(stop-int(start)):
            poz,piesa=moves[int(start)].split()
            #print(poz,piesa)
            #i=toate caracterele din poz mai putin ultimul, j= ultimul caracter
            i,j=poz[:-1],poz[-1]
            i=int(i)-1
            j=ord(j)-ord('A')
            #print (i,j)
            matrice_tabla[i][j]=int(piesa)
            if j - 2 >= 0 and j - 1 >= 0:  # Ensure indices are valid
                if piesa == matrice_tabla[i][j-2] + matrice_tabla[i][j-1] or piesa == matrice_tabla[i][j-2] * matrice_tabla[i][j-1] or piesa == abs(matrice_tabla[i][j-2] - matrice_tabla[i][j-1]) or piesa == max(matrice_tabla[i][j-2], matrice_tabla[i][j-1]) // max(min(matrice_tabla[i][j-2], matrice_tabla[i][j-1]), 1):
                    nr += 1

            if i - 2 >= 0 and i - 1 >= 0:  # Ensure indices are valid
                if piesa == matrice_tabla[i-2][j] + matrice_tabla[i-1][j] or piesa == matrice_tabla[i-2][j] * matrice_tabla[i-1][j] or piesa == abs(matrice_tabla[i-2][j] - matrice_tabla[i-1][j]) or piesa == max(matrice_tabla[i-2][j], matrice_tabla[i-1][j]) // max(min(matrice_tabla[i-2][j], matrice_tabla[i-1][j]), 1):
                    nr += 1

            if i + 2 < 14 and i + 1 < 14:  # Ensure indices are valid
                if piesa == matrice_tabla[i+2][j] + matrice_tabla[i+1][j] or piesa == matrice_tabla[i+2][j] * matrice_tabla[i+1][j] or piesa == abs(matrice_tabla[i+2][j] - matrice_tabla[i+1][j]) or piesa == max(matrice_tabla[i+2][j], matrice_tabla[i+1][j]) // max(min(matrice_tabla[i+2][j], matrice_tabla[i+1][j]), 1):
                    nr += 1

            if j + 2 < 14 and j + 1 < 14:  # Ensure indices are valid
                if piesa == matrice_tabla[i][j+2] + matrice_tabla[i][j+1] or piesa == matrice_tabla[i][j+2] * matrice_tabla[i][j+1] or piesa == abs(matrice_tabla[i][j+2] - matrice_tabla[i][j+1]) or piesa == max(matrice_tabla[i][j+2], matrice_tabla[i][j+1]) // max(min(matrice_tabla[i][j+2], matrice_tabla[i][j+1]), 1):
                    nr += 1



            scor+=int(piesa)*conf_speciala[i][j]
            scor*=max(nr,1)
        scoruri.append((player,start,scor))


    return scoruri



In [49]:
background = cv.imread(path_backgroud)
tabla = extrage_tabla(background)
tabla = cv.resize(tabla,(810,810))
careu_interior=extrage_careu_interior(tabla)
cv.imwrite("careu_interior.jpg",careu_interior)
linii, vert, oriz = extrage_linii_verticale_si_orizontale(careu_interior)

In [50]:
careu_interior =cv.medianBlur(careu_interior,3)
careu_interior = cv.GaussianBlur(careu_interior, (0, 0), 5)
careu_interior=cv.addWeighted(careu_interior, 1.5, careu_interior, -0.5, 0)

_,tresh = cv.threshold(careu_interior, 80, 255, cv.THRESH_BINARY_INV)
matrix = determina_configuratie_careu(tresh,oriz,vert)
matrice_pozitii_speciale = matrix.copy()

vizualizare_conf(careu_interior,matrix,oriz,vert)

# Task 1 si 2


In [51]:
background = cv.imread(path_backgroud)
tabla = extrage_tabla(background)
tabla = cv.resize(tabla,(810,810))
tabla=extrage_careu_interior(tabla)
tablafordiff = tabla.copy()
matrice_tabla = [[0 for i in range(15)] for j in range(15)]
matrice_tabla[6][6] = 1
matrice_tabla[6][7] = 2
matrice_tabla[7][6] = 3
matrice_tabla[7][7] = 4

folder = path_folder_joc
folder_destinatie=path_solutie
if not os.path.exists(folder_destinatie):
    os.makedirs(folder_destinatie)

for filename in os.listdir(folder):
    #daca este.jpg
    if filename.endswith(".jpg") :
        #extragem pozele
        img = cv.imread(os.path.join(folder,filename))
        img = cv.resize(img,(810,810))
        img = extrage_tabla(img)

        careu_interior=extrage_careu_interior(img)
        copy_of_image = careu_interior.copy()

        x,y,w,h=determina_pozitie_piesa_noua(tablafordiff,careu_interior)
        i,j = match_position(x,y,oriz,vert)
        piesa= extrage_piesa(i,j,copy_of_image)
        cifra= classify_patch(piesa)
        # print(cifra)
        matrice_tabla[i][j]=int(cifra)
        text=str(i+1)+chr(ord('A')+j)+" "+cifra+"\n"
        # print(filename, "poz:",chr(ord('a')+j),i+1)
        fisiernou=filename[:4]+".txt"

        #creaza un fisier cu numele filename.txt in folderul_destinatie

        with open(os.path.join(folder_destinatie, fisiernou), "w") as f:
            f.write(text)
            f.close()
        if filename.endswith("50.jpg"):
            tablafordiff = tabla.copy()
        else:
            tablafordiff = copy_of_image.copy()




# Task 3

In [52]:
matrice_tabla = [[0 for i in range(14)] for j in range(14)]
matrice_tabla[6][6] = 1
matrice_tabla[6][7] = 2
matrice_tabla[7][6] = 3
matrice_tabla[7][7] = 4

conf_speciala=[[1 for i in range(14)] for j in range(14)]
conf_speciala[0][0] = 3
conf_speciala[0][6] = 3
conf_speciala[0][7] = 3
conf_speciala[0][13] = 3
conf_speciala[6][0] = 3
conf_speciala[6][13] = 3
conf_speciala[7][0] = 3
conf_speciala[7][13] = 3
conf_speciala[13][0] = 3
conf_speciala[13][6] = 3
conf_speciala[13][7] = 3
conf_speciala[13][13] = 3

conf_speciala[1][1] = 2
conf_speciala[2][2] = 2
conf_speciala[3][3] = 2
conf_speciala[4][4] = 2
conf_speciala[9][9] = 2
conf_speciala[10][10] = 2
conf_speciala[11][11] = 2
conf_speciala[12][12] = 2
conf_speciala[12][1] = 2
conf_speciala[11][2] = 2
conf_speciala[10][3] = 2
conf_speciala[9][4] = 2
conf_speciala[4][9] = 2
conf_speciala[3][10] = 2
conf_speciala[2][11] = 2
conf_speciala[1][12] = 2






In [53]:
folder=path_folder_joc
folder_destinatie=path_solutie

ture=[]
moves=[]
for filename in os.listdir(folder):
    if filename.endswith("turns.txt") :
        linie=''
        with open(os.path.join(folder, filename), "r") as infile:
           for line in infile:
                player,start= line.strip().split()
                ture.append((player,start))
        scor = calculeaza_scor(ture,moves,matrice_tabla,conf_speciala)
        fisiernou=filename[0]+"_scores.txt"
        for line in scor:
            # print(line)
            linie=linie+line[0]+" "+line[1]+" "+str(line[2])+'\n'


        with open(os.path.join(folder_destinatie, fisiernou), "w") as f:
            f.write(linie)
            f.close()

        ture=[]
        moves=[]
    else:
        if filename.endswith(".jpg"):
            fisier=filename[:4]+".txt"
            # print(fisier)
            with open(os.path.join(folder_destinatie, fisier), "r") as infile:
                for line in infile:
                    moves.append(line.strip())



